In [ ]:
import numpy as np
from numpy import pi
from numpy.fft import rfft
import scipy.signal
from blindDescend import blindDescend
from matplotlib import pyplot as plt

import librosa
from IPython.display import Audio

In [ ]:
def init(frame_len, sr, upper_limit = 1600):
    global arange_size, hann, hann_half, \
    COEF_freq2Bin, PROD_freqWithCpesBin, \
    LEFT_TRIM, half_frame_len
    
    half_frame_len = frame_len // 2
    arange_size = np.arange(half_frame_len)
    hann = scipy.signal.get_window('hann', frame_len, True)
    hann_half = scipy.signal.get_window('hann', half_frame_len, True)
    COEF_freq2Bin = frame_len / sr
    PROD_freqWithCpesBin = sr / 2
    LEFT_TRIM = int(PROD_freqWithCpesBin / upper_limit) + 1

def getRotator(bin_period, size):
    coef = pi * 2j * bin_period / size
    return np.exp(arange_size * coef)

def sft(signal, size, bin):
    # Slow Fourier Transform
    return np.abs(np.sum(signal * getRotator(bin, size))) / size

def refineGuess(guess, guess_score, hanned_spectrum):
    def loss(x):
        return - sft(
            hanned_spectrum, half_frame_len, 
            x,
        )
    return blindDescend(loss, .1, .4, guess)[0]

def estimateF0(frame, frame_len, sr):
    hanned_spectrum = np.abs(rfft(frame * hann))[:-1] * hann_half
    cepstrum = np.abs(rfft(hanned_spectrum))
    guess = np.argmax(cepstrum[LEFT_TRIM:]) + LEFT_TRIM
    return PROD_freqWithCpesBin / refineGuess(
    guess, cepstrum[guess], hanned_spectrum
    )


In [ ]:
raw_i, sr = librosa.load('i.wav')
raw_a, sr = librosa.load('a.wav')

In [ ]:
sr

In [ ]:
def play(data):
    return Audio(data, rate = sr)
play(raw_a)

In [ ]:
FRAME_LEN = 2048
SR = sr
init(FRAME_LEN, SR)

In [ ]:
frame = raw_i[:FRAME_LEN]
frame_len = FRAME_LEN
sr = SR
estimateF0(frame, FRAME_LEN, SR)

In [ ]:
spectrum = np.abs(rfft(frame * hann))[:-1]
hanned_spectrum = spectrum * hann_half
cepstrum = np.abs(rfft(hanned_spectrum))

In [ ]:
plt.plot(spectrum[:180])

In [ ]:
220*COEF_freq2Bin

In [ ]:
LEFT = 3; RIGHT = 500

plt.plot(range(LEFT, RIGHT), cepstrum[LEFT:RIGHT] / half_frame_len)

x = np.linspace(LEFT, RIGHT, 1000)
y = [sft(hanned_spectrum, half_frame_len, b) for b in x]
plt.plot(x, y)

plt.gcf().set_size_inches(12,3)
plt.show()

In [ ]:
LEFT = 35; RIGHT = 65
plt.plot([PROD_freqWithCpesBin / x for x in range(LEFT, RIGHT)], cepstrum[LEFT:RIGHT] / half_frame_len)

x = np.linspace(PROD_freqWithCpesBin / LEFT, PROD_freqWithCpesBin / RIGHT, 1000)
y = [sft(hanned_spectrum, half_frame_len, PROD_freqWithCpesBin / f) for f in x]
plt.plot(x, y)

plt.gcf().set_size_inches(12,3)
# plt.xscale('log')
plt.show()

In [ ]:
play(raw_a)

In [ ]:
play([np.sin(2 * pi * 226 / sr * i) for i in range(sr)])

In [ ]:
play(raw_i)

In [ ]:
play([np.sin(2 * pi * 231 / sr * i) for i in range(sr)])